In [27]:
from transformers import (
    DebertaForSequenceClassification,
    EvalPrediction,
    DebertaConfig,
    DebertaTokenizer,
    Trainer,
    TrainingArguments,
    IntervalStrategy,
EarlyStoppingCallback
)
import pandas as pd
from sklearn.model_selection import train_test_split
import torch


In [3]:
df = pd.read_csv('text_sentiment_dataset.csv')


In [4]:
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {v: k for k, v in label2id.items()}

In [6]:
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")
def preprocess_function(examples):
    inputs = tokenizer(examples["Text"].values.tolist(), padding="max_length", truncation=True)
    inputs["labels"] = [label2id[label] for label in examples["Sentiment"].values]
    return inputs

In [10]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [15]:
class TextClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, examples):
        self.examples = examples 
    def __getitem__(self, index):
        return {k: torch.tensor(v[index]) for k, v in 
        self.examples.items()} 
    def __len__(self):
        return len(self.examples["input_ids"])
train_dataset = TextClassificationDataset(preprocess_function(train_df))
test_dataset = TextClassificationDataset(preprocess_function(test_df))

In [19]:
deberta_v3_config = {
    "model_type": "deberta-v2",
    "attention_probs_dropout_prob": 0.1,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "max_position_embeddings": 512,
    "relative_attention": True,
    "position_buckets": 256,
    "norm_rel_ebd": "layer_norm",
    "share_att_key": True,
    "pos_att_type": "p2c|c2p",
    "layer_norm_eps": 1e-7,
    "max_relative_positions": -1,
    "position_biased_input": False,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "type_vocab_size": 0,
    "vocab_size": 128100
}


In [20]:
deberta_config = {
    "model_type": "deberta-v2",
    "attention_probs_dropout_prob": 0.1,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "max_position_embeddings": 512,
    "relative_attention": True,
    "position_buckets": 256,
    "norm_rel_ebd": "layer_norm",
    "share_att_key": True,
    "pos_att_type": "p2c|c2p",
    "layer_norm_eps": 1e-7,
    "max_relative_positions": -1,
    "position_biased_input": False,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "type_vocab_size": 0,
    "vocab_size": 128100
}
model_config = DebertaConfig(id2label=id2label, label2id=label2id, **deberta_v3_config)
model = DebertaForSequenceClassification(model_config)

In [21]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [25]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1000,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy=IntervalStrategy.EPOCH,
    save_strategy=IntervalStrategy.EPOCH,
    load_best_model_at_end=True,
    learning_rate=0.000025,
    save_total_limit=2,
)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=20)]
    )

In [ ]:
trainer.train()
trainer_v2.evaluate()


KeyboardInterrupt



In [ ]:
from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(model, tokenizer)

In [ ]:
for idx in [0, 8, 6, 12]:
    text = test_df['Text'].iloc[idx]
    label = test_df['Sentiment'].iloc[idx]
    word_attributions = cls_explainer(text)
    cls_explainer.visualize()


In [ ]:
from transformers import TextClassificationPipeline
full_model = TextClassificationPipeline(model=model, tokenizer=tokenizer)
print(full_model("day"))
print(full_model("day day"))
print(full_model("day day day"))
print(full_model("day day day day"))
print(full_model("bored"))
print(full_model("bored bored"))
print(full_model("bored bored bored"))
print(full_model("bored bored bored bored"))